<a href="https://colab.research.google.com/github/AliaksandrSiarohin/articulated-animation/blob/main/demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Demo for paper "Motion Representations for Articulated Animation"
**Clone repository. First install git lfs. Pulling all checkpoints may take a while. 

If git lfs fails to download the checkpoint files, see cells below.**


In [ ]:
!curl -s https://packagecloud.io/install/repositories/github/git-lfs/script.deb.sh | sudo bash
!sudo apt-get install git-lfs
!git lfs install
!git clone https://github.com/snap-research/articulated-animation/

In [ ]:
cd articulated-animation

**Download the checkpoints, in case git LFS failed to download them from the GitHub repository. 
The following will work on Colab/Ubuntu, not on on Windows**.

[Source](https://stackoverflow.com/a/66804597).

In [4]:
%%capture
%%bash

 download_file_i(){
    wget --load-cookies /tmp/cookies.txt "https://drive.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=$1' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=$1" -O checkpoints/$2.pth && rm -rf /tmp/cookies.txt
 }

 a1=('1zt1cbLGiN6pVZi1Wn70lF29l1c2x6--V' '1UXMd2x4G3uFLa-XPA5iGywWlcidXR6Yg' 
    '1y_eiItgdKr3Z0Zb7lCsQoCM2GyxAcya_' '1X-BwTTwxTteesAxymrutxOy2qFW_jlWA'
    '1gyU9H7ArRAK9cUp_4rGF582C35B4pNcE')
a2=("mgif256" "taichi256" "ted-youtube384" "ted384" "vox256")

for i in "${!a1[@]}"; do
    download_file_i "${a1[i]}" "${a2[i]}"
done

`ffmpeg` is required to read the driving video file. 

In [10]:
!pip install imageio-ffmpeg

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


**Load source image and driving video.**

In [ ]:
import imageio
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from skimage.transform import resize
from IPython.display import HTML
import warnings
warnings.filterwarnings("ignore")

source_image = imageio.imread('sup-mat/source.png')
driving_video = imageio.mimread('sup-mat/driving.mp4')


source_image = resize(source_image, (384, 384))[..., :3]
driving_video = [resize(frame, (384, 384))[..., :3] for frame in driving_video]

def display(source, driving, generated=None):
    fig = plt.figure(figsize=(8 + 4 * (generated is not None), 6))

    ims = []
    for i in range(len(driving)):
        cols = [source]
        cols.append(driving[i])
        if generated is not None:
            cols.append(generated[i])
        im = plt.imshow(np.concatenate(cols, axis=1), animated=True)
        plt.axis('off')
        ims.append([im])

    ani = animation.ArtistAnimation(fig, ims, interval=50, repeat_delay=1000)
    plt.close()
    return ani
    

HTML(display(source_image, driving_video).to_html5_video())

**Load checkpoints.**

In [8]:
from demo import load_checkpoints

generator, region_predictor, avd_network = load_checkpoints(config_path='config/ted384.yaml',
                                                            checkpoint_path='checkpoints/ted384.pth')

**Perform animation.**

In [ ]:
from demo import make_animation
from skimage import img_as_ubyte

predictions = make_animation(source_image, driving_video, generator, 
                             region_predictor, avd_network, animation_mode='avd')

#save resulting video
imageio.mimsave('generated.mp4', [img_as_ubyte(frame) for frame in predictions])
#video can be downloaded from /content folder

HTML(display(source_image, driving_video, predictions).to_html5_video())